In [11]:
import keras
from keras.models import Sequential, Model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.merge import concatenate
import tensorflow as tf
from keras import backend as K

In [2]:
from keras.applications.vgg16 import VGG16

In [3]:
modelVGG16 = VGG16(include_top=False, weights='imagenet')
modelVGG16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [34]:
def get_VGG16_for_YOLO(IMAGE_H, IMAGE_W, NUMBER_OF_BBOXES, NUMBER_OF_CLASSES, weights='imagenet'):
    input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
    modelVGG16 = VGG16(include_top=False, weights=weights)
    VGG16out = modelVGG16(input_image)
    dense1 = Conv2D(512, kernel_size= (10,10), padding='same')(VGG16out)
    dense2 = Conv2D(512, kernel_size= (1,1), padding='same')(dense1)
    dense2 = Conv2D(NUMBER_OF_BBOXES * (4 + 1 + NUMBER_OF_CLASSES), kernel_size= (1,1), padding='same')(dense1)
    model = Model(inputs=input_image, outputs=dense2)
    return model

In [35]:
IMAGE_W = 320
IMAGE_H = 320
NUMBER_OF_BBOXES = 1
NUMBER_OF_CLASSES = 8
model_VGG = get_VGG16_for_YOLO(IMAGE_W, IMAGE_H, NUMBER_OF_BBOXES, NUMBER_OF_CLASSES)

In [36]:
model_aux.output

<tf.Tensor 'conv2d_24/BiasAdd:0' shape=(?, 10, 10, 13) dtype=float32>

In [5]:
layer_name = 'block4_pool'
last_VGG_layer = modelVGG16.get_layer(layer_name)


model = Model(modelVGG16.input, last_VGG_layer.output)

IMAGE_H = 320
IMAGE_W = 320
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
output = model(input_image)
model_aux = Model(inputs=input_image,
                                 outputs=model.get_output_at(1))

In [6]:
model_aux.output

<tf.Tensor 'model_1/block4_pool/MaxPool:0' shape=(?, 20, 20, 512) dtype=float32>

In [105]:
model.output

<tf.Tensor 'block4_pool_9/MaxPool:0' shape=(?, ?, ?, 512) dtype=float32>

In [62]:
IMAGE_H = 320
IMAGE_W = 320
input_image = Input(shape=(IMAGE_H, IMAGE_W, 3))
output = modelVGG16(input_image)
model = Model(input_image, output)
model.output

<tf.Tensor 'vgg16_15/block5_pool/MaxPool:0' shape=(?, 10, 10, 512) dtype=float32>